### Interactive Cropping Tool Demo

In [3]:
import cropping_widget
import os
image_list = ['examples/image_samples/'+i for i in os.listdir('examples/image_samples')]
image_names = [i[:-4].title() for i in os.listdir('examples/image_samples')]
cropping_widget.main(image_list, image_name_list=image_names, optimize=False,crop_shape= 'Ellipse')

GridBox(children=(Dropdown(description='Img Name:', layout=Layout(grid_area='im_selector', width='100%'), opti…